In [ ]:
import pickle, os

In [ ]:
import numpy as np

In [ ]:
import torch

In [ ]:
from datetime import datetime, timedelta

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import geopandas as gpd

In [ ]:
gdf = gpd.read_file('./../data/oxsfg_steel.gpkg').set_index("uid")

In [ ]:
root = os.path.abspath(os.path.join(os.getcwd(),'..'))

In [ ]:
results = pickle.load(open(os.path.join(root,'tmp','predictions.pkl'),'rb'))

In [ ]:
results[0][0]['y_hat'].shape

In [ ]:
tensors = {'y_hat':[],'y':[],'dates':[]}
for t in results[0]:
    if len(t['y_hat'].squeeze().shape)==2:
        tensors['y_hat'].append(t['y_hat'].squeeze())
    elif len(t['y_hat'].squeeze().shape)<2:
        tensors['y_hat'].append(t['y_hat'].squeeze().unsqueeze(0))
        
    if len(t['y'].squeeze().shape)==2:
        tensors['y'].append(t['y'].squeeze())
    elif len(t['y'].squeeze().shape)<2:
        tensors['y'].append(t['y'].squeeze().unsqueeze(0))
        
    if len(t['dates'].squeeze().shape)==2:
        tensors['dates'].append(t['dates'].squeeze().numpy())
    elif len(t['dates'].squeeze().shape)<2:
        tensors['dates'].append(t['dates'].squeeze().unsqueeze(0).numpy())

In [ ]:
all_dates = np.concatenate(tensors['dates'])

In [ ]:
y_hat = torch.cat(tensors['y_hat'])
y = torch.cat(tensors['y'])

In [ ]:
plt.hist(y_hat.numpy().flatten())

In [ ]:
plt.imshow(y.numpy())

In [ ]:
y.shape

In [ ]:
(y[:,0]<1).sum()

In [ ]:
y_hat[y[:,0]==1,0].mean() # => 1 -> there is a steel plant at t=0

In [ ]:
y_hat[y[:,0]<1,0].mean() # => 0 -> there is no steel plant at t=0

In [ ]:
y.shape

In [ ]:
y_hat.shape

In [ ]:
best_cuts = [(c,((y_hat>c)==y).sum().numpy()) for c in np.arange(0,1,0.01)]

In [ ]:
sorted(best_cuts, key=lambda k: k[1], reverse=True)[0]

In [ ]:
import pandas as pd

In [ ]:
true_idxs = [(i,j) for i,j in zip(*np.where((y_hat.numpy()>0.59)))]

In [ ]:
df = pd.DataFrame(true_idxs, columns=['row','col'])

In [ ]:
first_true = df.groupby('row').nth(0)

In [ ]:
((y_hat>)==y).sum()

In [ ]:
y.shape

In [ ]:
4804/57/100

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from matplotlib import cm

In [ ]:
cmap = cm.viridis

In [ ]:
y_hat_vis = cmap(y_hat.numpy())[:,:,:3]

In [ ]:
y_vis = y.numpy()

In [ ]:
import numpy as np

In [ ]:
start_dates = np.abs(y_vis - np.roll(y_vis, 1, axis=1))

In [ ]:
y_hat_vis[np.where(start_dates)] = np.array([1.,0.,0.])

In [ ]:
from matplotlib.lines import Line2D

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(12,6))
axs[0].imshow(y_hat_vis)
axs[1].imshow(y_vis)
axs[0].set_title('Prediction')
axs[1].set_title('Label')
axs[0].set_ylabel('Sample')
axs[0].set_xlabel('Date')
axs[1].set_xlabel('Date')
custom_lines = [Line2D([0], [0], color=cmap(0.), lw=4),
                Line2D([0], [0], color=cmap(1.), lw=4),
               Line2D([0], [0], color='#FF0000', lw=4)]
fig.legend(custom_lines, ['No Plant', 'Plant', 'Actual Startdate'], ncol=3, loc='lower center')
fig.savefig('./sobel_seq_testset.png')

In [ ]:
plt.imshow(y_hat.numpy(), vmax=1, vmin=0)

In [ ]:
plt.imshow(y.numpy())

In [ ]:
records = pickle.load(open('./../tmp/records.pkl','rb'))

In [ ]:
len(records)

In [ ]:
indices = pickle.load(open('./../tmp/indices.pkl','rb'))

In [ ]:
len(records[0]['revisits'])

In [ ]:
actual_dates = [int(gdf.loc[records[idx]['site'],'year']) for idx in indices['val']]

In [ ]:
actual_dates = [datetime(ii,1,1) for ii in actual_dates]

In [ ]:
len(indices['val'])

In [ ]:
indices['val'][0]

In [ ]:
est_start_dates = [all_dates[row, col] for row, col in df.groupby('row').nth(0)['col'].items()]

In [ ]:
est_start_dates = [datetime(1970,1,1)+timedelta(days=int(ii)) for ii in est_start_dates]

In [ ]:
len(actual_dates)

In [ ]:
len(est_start_dates)

In [ ]:
df_compare = pd.DataFrame({'act':actual_dates,'est':est_start_dates})

In [ ]:
df_compare['act'] = df_compare['act'].apply(lambda el: max(el, datetime(1984,1,1)))

In [ ]:
df_compare['diff'] = np.abs(df_compare['est'] - df_compare['act'])

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,6))
(df_compare.loc[df_compare['act']>datetime(1981,1,1),'diff'].dt.days/365).hist(ax=ax)
ax.set_xlabel('Years Difference')
ax.set_ylabel('Freq')
fig.savefig('./year_difference.png')

In [ ]:
df_compare['diff'].mean()

In [ ]:
df_compare['diff'].dt.days.hist()

In [ ]:
df_compare.loc[df_compare['act']<datetime(1990,1,1),:].shape

In [ ]:
df_compare.loc[(df_compare['act']<datetime(1990,1,1))&(df_compare['est']<datetime(1990,1,1)),:].shape

In [ ]:
df_compare['act'] = np.min(df_compare['act'], datetime(1984,1,1))

In [ ]:
df

In [ ]:
est_start_dates